<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_ShipCrew_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>
<hr>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Calcutta')))

Tested 2024-07-03 09:18:28.757103+05:30


# BDSN Ship Crews <br>
https://www.geeksforgeeks.org/pyspark-linear-regression-using-apache-mllib/

# Initialise

## Deprecated -- do not use

In [2]:
#!apt-get update > /dev/null
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#!pip install -q pyspark

In [3]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

## Install Spark, current version

In [4]:
#import findspark
#findspark.init()
!pip3 install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praxis_Shipping').master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


<SparkContext master=local[*] appName=Praxis_Shipping>

# Load Data <br>
Data is available at https://drive.google.com/file/d/1fLGDQjsnA3RzTzzEVDpHxyy5ERzcV9Qw/view?usp=sharing

In [5]:
# Get Data
#!gdown https://drive.google.com/uc?id=1fLGDQjsnA3RzTzzEVDpHxyy5ERzcV9Qw
!wget -q 'https://raw.githubusercontent.com/Praxis-QR/BDSN/main/data/ML/ShippingCrewInfo.csv'



In [6]:
#!ls -al

In [7]:
crew_df=spark.read.csv('/content/ShippingCrewInfo.csv',inferSchema=True,header=True)
crew_df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
#prints structure of dataframe along with datatype
crew_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



# Pre Processing : Indexing, Vectorising

In [9]:
#columns identified as features are as below:
#['Cruise_line','Age','Tonnage','passengers','length','cabins','passenger_density']
#to work on the features, spark MLlib expects every value to be in numeric form
#feature 'Cruise_line is string datatype
#using StringIndexer, string type will be typecast to numeric datatype
#import library strinindexer for typecasting

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
crew_df2 = indexer.fit(crew_df).transform(crew_df)                        # indexer.fit(crew_df) is a Transformer

#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model
crew_df2.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
crew_df3=assembler.transform(crew_df2)                              # assember is a Transformer
crew_df3.head(3)
#crew_df3.select('features','crew').show(10)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0, features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0]))]

In [11]:
#final data consist of features and label which is crew.
data=crew_df3.select('features','crew')
#splitting data into train and test
train_data,test_data=data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               108|
|   mean| 7.663888888888899|
| stddev|3.5464972408177236|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



# Regression
Read this https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html

In [12]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
#pass train_data to train model
trained_ship_model=ship_lr.fit(train_data)                     # trained_ship_model, a TRAINED model, is a Transformer
#evaluating model trained for Rsquared error
ship_results=trained_ship_model.evaluate(train_data)

print('Rsquared Error :',ship_results.r2)
#R2 value shows accuracy of model is 92%
#model accuracy is very good and can be use for predictive analysis

Rsquared Error : 0.9631240296093722


## Prediction

### With Unlabelled Data

In [13]:
#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[6.0,93.0,23.94,9...|
|[6.0,112.0,38.0,9...|
|[6.0,113.0,37.82,...|
|[7.0,116.0,31.0,9...|
+--------------------+
only showing top 5 rows



In [14]:
predictions=trained_ship_model.transform(unlabeled_data)      # the Transformer, trained_ship_model, is transforming data, unlabelled_data
predictions.show()
#below are the results of output from test data

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.184052305041378|
|[6.0,93.0,23.94,9...| 10.39424962854118|
|[6.0,112.0,38.0,9...| 11.12926453176487|
|[6.0,113.0,37.82,...| 11.46482673052301|
|[7.0,116.0,31.0,9...|12.396928501494902|
|[8.0,91.0,22.44,9...| 9.973799992475103|
|[9.0,110.0,29.74,...|11.880535456324269|
|[9.0,113.0,26.74,...|11.227368441815727|
|[9.0,116.0,26.0,9...|11.091251474081988|
|[10.0,58.825,15.6...| 7.192618787754309|
|[10.0,77.0,20.16,...| 8.650554619929116|
|[10.0,110.0,29.74...|11.867608038857785|
|[11.0,90.0,22.4,9...| 9.922009532334204|
|[11.0,91.62700000...| 9.192093997055846|
|[11.0,110.0,29.74...| 11.87467413157462|
|[11.0,138.0,31.14...|12.890559360820149|
|[12.0,25.0,3.88,5...|3.0360934694129003|
|[12.0,58.6,15.66,...|  7.30385647309023|
|[12.0,88.5,21.24,...|10.213471947743997|
|[12.0,91.0,20.32,...| 9.191730936201672|
+--------------------+------------

### With Labelled Data

In [15]:
# Generate predictions
predicted2 = trained_ship_model.transform(test_data)          # the Transformer, trained_ship_model, is transforming data, test_data
predicted2.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[5.0,86.0,21.04,9...|  8.0| 9.184052305041378|
|[6.0,93.0,23.94,9...|11.09| 10.39424962854118|
|[6.0,112.0,38.0,9...| 10.9| 11.12926453176487|
|[6.0,113.0,37.82,...| 12.0| 11.46482673052301|
|[7.0,116.0,31.0,9...| 12.0|12.396928501494902|
|[8.0,91.0,22.44,9...| 11.0| 9.973799992475103|
|[9.0,110.0,29.74,...| 11.6|11.880535456324269|
|[9.0,113.0,26.74,...|12.38|11.227368441815727|
|[9.0,116.0,26.0,9...| 11.0|11.091251474081988|
|[10.0,58.825,15.6...|  7.0| 7.192618787754309|
|[10.0,77.0,20.16,...|  9.0| 8.650554619929116|
|[10.0,110.0,29.74...| 11.6|11.867608038857785|
|[11.0,90.0,22.4,9...| 11.0| 9.922009532334204|
|[11.0,91.62700000...|  9.0| 9.192093997055846|
|[11.0,110.0,29.74...| 19.1| 11.87467413157462|
|[11.0,138.0,31.14...|11.85|12.890559360820149|
|[12.0,25.0,3.88,5...| 2.87|3.0360934694129003|
|[12.0,58.6,15.66,...|  7.0|  7.30385647

In [16]:
trained_ship_model.setPredictionCol("PraxisPredictions!")

LinearRegressionModel: uid=LinearRegression_1139feb839ef, numFeatures=7

In [17]:
# Generate predictions
predicted3 = trained_ship_model.transform(test_data)
predicted3.show()

+--------------------+-----+------------------+
|            features| crew|PraxisPredictions!|
+--------------------+-----+------------------+
|[5.0,86.0,21.04,9...|  8.0| 9.184052305041378|
|[6.0,93.0,23.94,9...|11.09| 10.39424962854118|
|[6.0,112.0,38.0,9...| 10.9| 11.12926453176487|
|[6.0,113.0,37.82,...| 12.0| 11.46482673052301|
|[7.0,116.0,31.0,9...| 12.0|12.396928501494902|
|[8.0,91.0,22.44,9...| 11.0| 9.973799992475103|
|[9.0,110.0,29.74,...| 11.6|11.880535456324269|
|[9.0,113.0,26.74,...|12.38|11.227368441815727|
|[9.0,116.0,26.0,9...| 11.0|11.091251474081988|
|[10.0,58.825,15.6...|  7.0| 7.192618787754309|
|[10.0,77.0,20.16,...|  9.0| 8.650554619929116|
|[10.0,110.0,29.74...| 11.6|11.867608038857785|
|[11.0,90.0,22.4,9...| 11.0| 9.922009532334204|
|[11.0,91.62700000...|  9.0| 9.192093997055846|
|[11.0,110.0,29.74...| 19.1| 11.87467413157462|
|[11.0,138.0,31.14...|11.85|12.890559360820149|
|[12.0,25.0,3.88,5...| 2.87|3.0360934694129003|
|[12.0,58.6,15.66,...|  7.0|  7.30385647

In [18]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2024-07-03 09:20:11.429776+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)